# This notebook is to take the union of peaks

In [1]:
from pybedtools import BedTool
peakdir='/home/hsher/YTHDF2/raw_beds/'

condition_dict = {'YTHDF2':
                  {'lib': ['m1', 'm2', 'm3'],
                  'fast':['m4', 'm5', 'm6']
                 },
                  'm6A':
                {'lib': ['m7', 'm8', 'm9'],
                  'fast':['m10', 'm11', 'm12']
                 }}

def read_3_rep(rbp, condition):
    three_peaks = []
    for lib in condition_dict[rbp][condition]: #['m1', 'm2', 'm3']
        fname = 'zt2_liver_{}_eCLIP.{}_IP.umi.r1.fq.genome-mappedSoSo.rmDupSo.peakClusters.normed.compressed.sort.bed'.format(rbp, lib)
        three_peaks.append(BedTool(peakdir+fname))
    return three_peaks

In [2]:
from collections import Counter
# consensus peaks
def get_consensus(peak_list):
    ''' does not have strand information!'''
    x = BedTool()
    result = x.multi_intersect(i=[p.fn for p in peak_list])
    
    return(result)
def count_consensus(result):
    return Counter([r[3] for r in result])

# Calculate how well they overlap

In [3]:
stat = []
for rbp in ['YTHDF2', 'm6A']:
    for cond in ['lib', 'fast']:
        

        beds = read_3_rep(rbp, cond)
        indv_no = [len(bed) for bed in beds]
        result = get_consensus(beds)
        intersect_cond = count_consensus(result)
        stat.append([rbp, cond]+indv_no+list(intersect_cond.values()))


In [4]:
import pandas as pd
stat_df = pd.DataFrame(stat, columns = ['rbp', 'cond', '#peak1', '#peak2', '#peak3', 'only1rep', 'only2rep', 'only3rep'])

In [5]:
stat_df

,rbp,cond,#peak1,#peak2,#peak3,only1rep,only2rep,only3rep
0,YTHDF2,lib,14083,33297,36567,66137,25326,7011
1,YTHDF2,fast,30725,37097,49947,88162,39740,14016
2,m6A,lib,153529,139482,26745,200876,101033,16195
3,m6A,fast,126405,41794,116714,175019,97665,30073


# Union of three reps, sort and merge

In [6]:
outdir = '/home/hsher/YTHDF2/intersecting_beds/'
import os
def either_rep_peak(peak_list, output_fname):
    os.system('cat {} {} {} > {}concat.bed'.format(peak_list[0].fn, peak_list[1].fn, peak_list[2].fn, outdir))
    os.system('bedtools sort -i {}concat.bed > {}sorted.bed'.format(outdir, outdir))
    
    
    
    os.system('bedtools merge -i {}sorted.bed -s -c 4,5,6 -o min,max,distinct > {}{}'.format(outdir, outdir, output_fname))
    os.system('rm {}concat.bed'.format(outdir))
    os.system('rm {}sorted.bed'.format(outdir))

In [7]:
stat = []
for rbp in ['YTHDF2', 'm6A']:
    for cond in ['lib', 'fast']:
        

        beds = read_3_rep(rbp, cond)
        fname = '{}_{}_union_bed6.bed'.format(rbp, cond)
        either_rep_peak(beds, fname)
        print(fname)

YTHDF2_lib_union_bed6.bed
YTHDF2_fast_union_bed6.bed
m6A_lib_union_bed6.bed
m6A_fast_union_bed6.bed


# find regions supported by at least 2 reps

In [9]:
print(result[0])

chr1	3531677	3531697	1	2	0	1	0



In [18]:
def at_least_two_rep(rbp, cond):
    ''' bedtool intersect to get list of regions with at least 2 reps'''
    bedlist = read_3_rep(rbp, cond)
    
    # multiintersect does not take care of strandedness
    first = bedlist[0].intersect(bedlist[1], u = True, s = True)
    second = bedlist[1].intersect(bedlist[2], u = True, s = True)
    third = bedlist[2].intersect(bedlist[0], u = True, s = True)
    
    
    
    # concat
    cat = first.cat(second, postmerge = False).cat(third, postmerge = False).sort().merge(s = True, c='4,5,6',o='min,max,distinct').saveas('{}{}_{}_2rep_bed6.bed'.format(outdir, rbp, cond))
    
    return cat
    
    
    

In [20]:
stat = []
for rbp in ['YTHDF2', 'm6A']:
    for cond in ['lib', 'fast']:
        c = at_least_two_rep(rbp, cond)